In [1]:
def basis_reconciliation(alice, bob):
    """
        alice: {no. : [bit encoded, basis chosen to encode in ]}
        bob  : {no. : [bit after measurement, basis chosen to measure in]}
        
        First check if the length of both lists are the same
            -> if yes, keep only those bits for alice and bob for which
               the basis encoded in and measured in is the same. 
    """
    basis_bit_alice = list(alice.values())
    basis_bit_bob = list(bob.values())
    
    if len(basis_bit_alice) == len(basis_bit_bob):
        raw_key_alice = []
        raw_key_bob = []

        for i in range(len(basis_bit_alice)):
            if basis_bit_alice[i][1] == basis_bit_bob[i][1]:
                raw_key_alice.append(basis_bit_alice[i][0])
                raw_key_bob.append(basis_bit_bob[i][0])

        return raw_key_alice, raw_key_bob
    
    else:
        return None, None

In [2]:
alice = {1:[0,0],2:[0,1],3:[0,0]}
bob = {1:[1,0],2:[0,0],3:[1,0]}

In [3]:
basis_reconciliation(alice, bob)

([0, 0], [1, 1])

In [4]:
class Alice:
    def __init__(self, n):
        self.n = n
        self.alice = {} #{no. : [bit encoded, basis chosen to encode in ]} no. is some unique number
                        #{1:[0,0],2:[0,1],3:[0,0]} Example

    def generate_and_encode(self): #TODO Dighvijay
        """
            Will generate n bits randomly
            For each bit generated, a basis is chosen in which it is encoded
            Dependency for encoding: <class LinearPolarizer>
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.alice mentioned above
        """
        pass
    

class Bob:
    def __init__(self, n):
        self.n = n
        self.bob = {} #{no. : [bit after measurement, basis chosen to measure in]}
                      #{1:[1,0],2:[0,0],3:[1,0]} Example
    
    def choose_basis_and_measure(self, received): #TODO Dighvijay
        """
            received : the data received by bob
            Dependency for measurement: <class PolarizingBeamSplitter>
            
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.bob mentioned above
        """
        pass
    
    

In [5]:
def calc_error_rate(alice_b, bob_b):
    error = 0
    count = 0
    for j in range(0, len(alice_b), 4):
        i = j-count
        #i = alice_b.index(j)
        if(alice_b[i]!=bob_b[i]):
            error+=1
        count+=1
        alice_b.pop(i)
        #alice.pop(i)
        bob_b.pop(i)
        #bob.pop(i)
    error_rate = float(error/count)
    return error, count, error_rate

def set_length(error, count):
    s_len = int(count/error)
    if s_len<3:
        return 3
    return s_len
def find_parity(bits):
    count = 0
    for i in bits:
        count+=i
    par = count%2
    return par

def remove_last(bits):
    bits.pop()
    return bits


def binary_search(alice_sub, bob_sub):
    r = len(alice_sub)
    l = 0
    
    while(l < r):
        alice = alice_sub[l:r]
        bob = bob_sub[l:r]
        m = int((r + l)/2)        
        if (find_parity(alice) != find_parity(bob)):
            l = m + 1
        else:
            r = m - 1

    bob_sub[m] = int(not bob_sub[m])
    
    return bob_sub

def get_subsets(error, count, alice_b, bob_b):
    alice, bob = [], []
    n = len(alice_b)
    subset_size = set_length(error, count)
    for i in range(0, n, subset_size):
        alice.append(alice_b[i:i+subset_size])
        bob.append(bob_b[i:i+subset_size])
        
    return alice, bob

def error_correction(alice_b, bob_b, error, count):
    alice_sub, bob_sub = [], []
    alice_sub, bob_sub = get_subsets(error, count, alice_b, bob_b)
    par_alice, par_bob = [], []
    for i in range(len(alice_sub)):
        par_alice.append(find_parity(alice_sub[i]))
        alice_sub[i].pop(0)
        par_bob.append(find_parity(bob_sub[i]))
        bob_sub[i].pop(0)

    corrected_alice, corrected_bob = [], []
    for i in range(len(par_alice)):
        if par_alice[i]==par_bob[i]:
            corrected_bob.append(bob_sub[i])
        else:
            corrected_bob.append(binary_search(alice_sub[i], bob_sub[i]))
        corrected_alice.append(alice_sub[i])
    return corrected_alice, corrected_bob
    


#Selecting set length
#Formula: n-k-s
#n: total number of bits,
#k: estimated maximum number of bits known by eve (double the error rate)
#s: security parameter
#parities of these subsets becomes final key

"""
k = int(2*error) + 1
n = len(alice_b)
s = n - 2k
if s>7 or s<0:
    s = 4
for i in range()
"""

def privacy_amplification(n, error_rate, s, alice_bit, bob_bit):
    k = int(error_rate * 2) 
    subset_size = n - k - s
    final_alice = []
    final_bob = []
    
    alice_b = []
    bob_b = []
    
    alice_subsets = []
    bob_subsets = []
    
    for i in alice_bit:
        for j in i:
            alice_b.append(j)
    
    for i in bob_bit:
        for j in i:
            bob_b.append(j)
    
    for i in range(0, n, subset_size):
        alice_subsets.append(alice_b[i:i+subset_size])
        bob_subsets.append(bob_b[i:i+subset_size])
        
    bob_parity = 0
    alice_parity = 0
    
    #calculate parities of sets and compare and eliminate if parities dont match
    for i in range(len(alice_subsets)):
        
        alice = find_parity(alice_subsets[i])
        bob = find_parity(bob_subsets[i])
                
        if alice == bob:
            final_alice.append(alice)
            final_bob.append(bob)
    
    return final_alice, final_bob

In [6]:
import numpy as np
import random
import copy
from itertools import chain

In [7]:
class LinearPolarizer:
    def __init__(self):
        
        self.x = np.array([[1], [0]])
        self.y = np.array([[0], [1]])
    
    def horizontal_vertical(self, bit):
        if bit == 0:
            return self.x
        else:
            return self.y
    
    def diagonal_polarization(self, bit):
        jones = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])
        
        if bit == 0:
            return np.dot(jones, self.x)
        else:
            return np.dot(jones, self.y)
    
    def general_polarization(self, angle, basis):
        """
            angle to be in degrees
        """
        angle = (math.pi/180) * (angle)
        jones = np.array([[np.cos(angle), np.sin(angle)], [np.sin(angle), -np.cos(angle)]])
        
        return np.dot(jones, basis)

class PolarizingBeamSplitter:
    def __init__(self):
        pass
    
    def measure(self, vector, basis):
        """
            basis  : basis chosen by bob to measure polarization encoded photon
                        0 -> horizontal/vertical 
                        1 -> diagonal
            vector : Jones vector for polarized photon
            
            returns a dictionary with probabilities of the encoded bit sent by Alice being 0 or 1
        """
        #horizontal-vertical can be clubbed into an identity matrix
        horizontal = np.array([[1, 0], [0, 0]])
        vertical = np.array([[0, 0], [0, 1]])
        plus_minus = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])

        if basis == 0:
            zero = np.dot(horizontal, vector)[0]
            one = np.dot(vertical, vector)[1]
        
        elif basis == 1:
            zero = np.dot(plus_minus, vector)[0]
            one = np.dot(plus_minus, vector)[1]
        else:
            print("here")
            return None
            
        return {0: zero[0]**2, 1: one[0]**2}
class Alice:
    def __init__(self, n):
        self.n = n
        self.alice = {} #{no. : [bit encoded, basis chosen to encode in ]} no. is some unique number
                        #{1:[0,0],2:[0,1],3:[0,0]} Example

    def generate_and_encode(self): 
        """
            Will generate n bits randomly
            For each bit generated, a basis is chosen in which it is encoded
            Dependency for encoding: <class LinearPolarizer>
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.alice mentioned above
        """
        LP = LinearPolarizer()
        encode = []
        
        while self.n!= 0:
            self.alice[self.n] = [ random.randint(0,1), random.randint(0,1)]
            if self.alice[self.n][1] == 0:
                encode.append(LP.horizontal_vertical(self.alice[self.n][0]))
            else:
                encode.append(LP.diagonal_polarization(self.alice[self.n][0]))
            self.n-=1
        
        return encode
    

class Bob:
    def __init__(self, n):
        self.n = n
        self.bob = {} #{no. : [bit after measurement, basis chosen to measure in]}
                      #{1:[1,0],2:[0,0],3:[1,0]} Example
    
    def choose_basis_and_measure(self, received):
        """
            received : the data received by bob
            Dependency for measurement: <class PolarizingBeamSplitter>
            
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.bob mentioned above
        """
        #self.bob[n][0] is the measured bit
        
        PBS = PolarizingBeamSplitter()
        
        while self.n!= 0:
            i = self.n
            self.bob[self.n] = [0, random.randint(0,1)]
            self.bob[self.n][0] = random.randint(0,1) if (PBS.measure(received[i-self.n], self.bob[self.n][1])[0] == PBS.measure(received[i-self.n], self.bob[self.n][1])[1]) else (0 if (PBS.measure(received[i-self.n], self.bob[self.n][1])[0]> PBS.measure(received[i-self.n], self.bob[self.n][1])[1]) else 1)
            #Here,  picking randomly between 0 and 1 if wrong basis is choosen else 0 or 1 based of actual measurement
            self.n-=1
        


In [8]:
alice = Alice(10)
encoded = alice.generate_and_encode()

In [9]:
bob = Bob(10)
bob.choose_basis_and_measure(encoded)

In [10]:
recon_alice, recon_bob = basis_reconciliation(alice.alice, bob.bob)
temp_alice_r = copy.copy(recon_alice)
temp_bob_r = copy.copy(recon_bob)

In [11]:
recon_alice

[1, 1, 1, 1, 1, 1]

In [12]:
recon_bob

[1, 1, 1, 0, 1, 0]

In [13]:
error, count, error_rate = calc_error_rate(temp_alice_r, temp_bob_r)

In [14]:
corrected_a, corrected_b = error_correction(recon_alice, recon_bob, error, count)
corrected_a, corrected_b

ZeroDivisionError: division by zero

In [ ]:
privacy_amplification(len(corrected_a)*len(corrected_a[0]), error_rate, 2, corrected_a, corrected_b)

In [ ]:
alice1 = Alice(100)
encoded1 = alice1.generate_and_encode()
bob1 = Bob(100)
bob1.choose_basis_and_measure(encoded1)

In [ ]:
recon_alice1, recon_bob1 = basis_reconciliation(alice1.alice, bob1.bob)
temp_alice_r1 = copy.copy(recon_alice1)
temp_bob_r1 = copy.copy(recon_bob1)

In [ ]:
error1, count1, error_rate1 = calc_error_rate(temp_alice_r1, temp_bob_r1)

In [ ]:
corrected_a1, corrected_b1 = error_correction(recon_alice1, recon_bob1, error1, count1)
str(list(chain.from_iterable(corrected_a1))), str(list(chain.from_iterable(corrected_b1)))